<a href="https://colab.research.google.com/github/lavData/ltss_seminar/blob/main/Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Concurrent Linear Hashing

```
# This is formatted as code
```



# Linear Hashing

### What is Linear Hashing?
---

Linear Hashing là một kỹ thuật băm (hashing) cho phép không gian địa chỉ có thể tăng hoặc giảm động dựa trên lượng dữ liệu được lưu trữ. Điều này cho phép tệp tin hoặc bảng có thể hỗ trợ bất kỳ số lượng thêm vào hoặc xóa bỏ nào mà không ảnh hưởng đến hiệu suất truy cập hay sử dụng bộ nhớ. Trong Linear Hashing, một bản ghi trong tệp tin hoặc bảng có thể được tìm thấy trong một thao tác truy cập, trong khi đó tải (load) có thể duy trì ở mức gần 90%. Trong trường hợp bảng, một bản ghi trung bình có thể được tìm thấy trong 1.7 lần truy cập, trong khi tải luôn ổn định ở mức 80%. Hiện tại, không có thuật toán nào khác có thể đạt được hiệu suất tốt như vậy.

Các cấu trúc dữ liệu cơ bản nhất là các tệp tin (files) và các bảng (tables) được xác định bởi một khóa chính (primary key). Trong đó, kỹ thuật băm (hashing) và các cây (B-tree, binary tree,..) là các kỹ thuật địa chỉ cơ bản cho các tệp tin và bảng này. Có hàng nghìn công trình nghiên cứu đã giải quyết vấn đề này.

Nếu một tệp tin hoặc bảng gần như tĩnh, thì kỹ thuật băm cho phép tìm thấy một bản ghi trong một lần truy cập. Một cây luôn yêu cầu nhiều lần truy cập. Tuy nhiên, khi tệp tin hoặc bảng, như thường lệ, có tính động, thì cây vẫn hoạt động tương đối tốt, trong khi hiệu suất của kỹ thuật băm có thể trở nên rất kém. Thậm chí có thể cần phải thực hiện lại toàn bộ quá trình băm để đưa tất cả các bản ghi vào một tệp tin mới. 

Linear Hashing là một cấu trúc chỉ mục (index structure) dựa trên đĩa, cho phép cập nhật, mở rộng hoặc thu nhỏ một thùng (bucket) một cách linh hoạt (dynamically), hay còn gọi là một kỹ thuật băm động (dynamic hashing scheme). Cấu trúc chỉ mục này được sử dụng để tìm kiếm bản ghi tương ứng với một khóa cụ thể hoặc để tạo thuận tiện cho việc tìm kiếm trùng khớp chính xác.

Phương pháp này được gọi là Linear Hashing bởi vì số lượng thùng tăng hoặc giảm một cách tuyến tính. Số lượng hàm băm mà hệ thống có thể sử dụng cùng một lúc thay đổi động.

### The distinction between linear hashing and other hashing

---



- Cấu trúc Linear Hashing không yêu cầu sử dụng thư mục (directory) để lưu trữ các bản ghi, giúp giảm thiểu độ phức tạp và tăng tốc độ truy cập.

- Linear Hashing có khả năng xử lý các chuỗi tràn (overflow chains) dài, giúp tối ưu hóa sử dụng không gian lưu trữ và tránh tình trạng tràn.

- Linear Hashing linh hoạt hơn về thời điểm chia (split) các thùng, giúp tối ưu hóa hiệu suất và giảm thiểu khả năng xảy ra tình trạng tràn.

- Linear Hashing cho phép chúng ta mở rộng một slot (khe) một cách linh hoạt, giúp tối ưu hóa sử dụng không gian lưu trữ và giảm thiểu khả năng xảy ra tình trạng tràn.

In [18]:
from numba import njit, prange, types, typed, int64, float64, boolean
import random

class LinearHash:
    def __init__(self, size=2, threshold=0.75):
        self.size = size
        self.threshold = threshold
        self.count = 0
        self.data = typed.List()
        for i in range(self.size):
            self.data.append(typed.List.empty_list(types.Tuple((int64, float64))))

    @njit(parallel=True)
    def hash_func(self, key):
        return hash(key) % self.size

    @njit(parallel=True)
    def add(self, key, value):
        if self.count >= self.size * self.threshold:
            self.size *= 2
            self.rehash()

        index = self.hash_func(key)
        bucket = self.data[index]
        for i in prange(len(bucket)):
            if bucket[i][0] == key:
                bucket[i] = (key, value)
                return
        bucket.append((key, value))
        self.count += 1

    @njit(parallel=True)
    def get(self, key):
        index = self.hash_func(key)
        bucket = self.data[index]
        for i in prange(len(bucket)):
            if bucket[i][0] == key:
                return bucket[i][1]
        return None

    @njit(parallel=True)
    def remove(self, key):
        index = self.hash_func(key)
        bucket = self.data[index]
        for i in prange(len(bucket)):
            if bucket[i][0] == key:
                bucket.pop(i)
                self.count -= 1
                return
        raise KeyError("Key not found")

    @njit(parallel=True)
    def rehash(self):
        new_data = typed.List()
        for i in range(self.size):
            new_data.append(typed.List.empty_list(types.Tuple((int64, float64))))

        for bucket in self.data:
            for key, value in bucket:
                index = self.hash_func(key)
                new_data[index].append((key, value))
        self.data = new_data

    def __str__(self):
        result = ""
        for bucket in self.data:
            for key, value in bucket:
                result += str(key) + " -> " + str(value) + "\n"
        return result

In [19]:
lh = LinearHash()
lh.add(1, 100)
lh.add(2, 200)
lh.add(3, 300)

print(lh.get(1)) # in ra 100
print(lh.get(2)) # in ra 200
print(lh.get(3)) # in ra 300

lh.remove(2)
print(lh.get(2)) # in ra None

TypingError: ignored

# References
- https://www.educative.io/answers/how-to-implement-linear-hashing-in-python
- https://www.cs.bu.edu/faculty/gkollios/ada17/LectNotes/linear-hashing.PDF